In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4395504 sha256=af40d04285e3aef8d2cd6b24e3207dad08d17608faeb0c41bf08ab9a0a094305
  Stored in directory: /root/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pygeohash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6164 sha256=5ff3e745c1eb18b7c484ca401e5a284c08170f90d98214fe62ab0d7302aa0d60
  Stored in directory: /root/.cache/pip/wheels/6e/e7/a2/4ea56930e1cc3ece852306b842f9ca71e0cd9482cb8b848e27
Successfully built pygeohash


In [ ]:
!pip install git+https://github.com/RDFLib/sparqlwrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/RDFLib/sparqlwrapper to /tmp/pip-req-build-e7pfhfsh
  Running command git clone --filter=blob:none --quiet https://github.com/RDFLib/sparqlwrapper /tmp/pip-req-build-e7pfhfsh
  Resolved https://github.com/RDFLib/sparqlwrapper to commit 2a6e2d3ddbc3fe38ca47d6d05f23c9b61ff82366
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00
  Created wheel for SPARQLWrapper: filename=SPARQLWrapper-2.0.1a0-py3-none-any.whl size=28648 sha256=8b3115e1467139cb649ec48b213ac2cc9c0a4942bada0dcb40a2370509010f63
  Stored in directory: /tmp/pip-ephem-wheel-cache-vupyq8sh/wheels/64/db/31/75e25543c73b964f8882469c29b935f092182b765b6a0beb9f
Success

In [ ]:
import glob
import os.path
import re
import csv


import pandas as pd
import numpy as np

from urllib.error import HTTPError, URLError
from SPARQLWrapper import SPARQLWrapper, JSON, POST, GET, POSTDIRECTLY, CSV

from io import StringIO
import math
from tqdm import tqdm
import time

import sys
import pickle

import pygeohash as gh
from haversine import haversine_vector, Unit
import h5py
import fasttext 
import fasttext.util
import pdb
from sklearn.model_selection import train_test_split

## USLP Data Preparation

In [ ]:
# test triples
test_triples_file_name = 'test.txt'
test_triples = pd.read_csv(test_triples_file_name, sep = '\t', names = ['subject', 'predicate', 'object'])
test_triples

,subject,predicate,object
0,wkg:4426850370,wkgs:isInCountry,wkg:424298311
1,wkg:3068656670,wkgs:isInCountry,wkg:424313730
2,wkg:630639045,wkgs:isInContinent,wkg:36966065
3,wkg:4426852196,wkgs:isInContinent,wkg:36966065
4,wkg:2000003612,wkgs:isInCountry,wkg:424298311
...,...,...,...
15926,wkg:274372610,wkgs:addrCountry,wkg:1683325355
15927,wkg:1254881907,wkgs:addrCountry,wkg:424314830
15928,wkg:1261054393,wkgs:addrCountry,wkg:424314830
15929,wkg:279000168,wkgs:addrCountry,wkg:6907263340


In [ ]:
# unique entities in inductive test set
unique_entities_inductive_testset = np.unique(test_triples[['subject', 'object']].values)
len(unique_entities_inductive_testset)

16190

In [ ]:
unique_entities_inductive_testset

array(['wkg:1003705119', 'wkg:1011817361', 'wkg:1012761434', ...,
       'wkg:997414904', 'wkg:997414940', 'wkg:997415479'], dtype=object)

## Fetch inductive split unique entities data using SPARQL

In [ ]:
unique_entities_inductive_testset.shape

(16190,)

In [ ]:
# SPARQL Extraction Utilities

def create_sparql_client(sparql_endpoint, http_query_method = GET, result_format= JSON):
    sparql_client = SPARQLWrapper(sparql_endpoint)

    sparql_client.setMethod(http_query_method)
    sparql_client.setReturnFormat(result_format)

    return sparql_client

# Convert SPARQL results into a Pandas data frame
def sparql2dataframe(json_sparql_results):
    cols = json_sparql_results['head']['vars']
    out = []
    for row in json_sparql_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

def query_sparql(query, sparql_client):
    sparql_client.setQuery(query)
    result_object = sparql_client.query()
    if sparql_client.returnFormat == JSON:
        return result_object._convertJSON()
    return result_object.convert()

In [ ]:
sparqlview_endpoint = "https://www.worldkg.org/sparql"
sparqlview_wrapper = create_sparql_client(sparql_endpoint=sparqlview_endpoint)

In [ ]:
# extract all triples for each unique entity
batch_size = 100
batches = math.ceil(len(unique_entities_inductive_testset)/batch_size)

results_df_list = []
for i in tqdm(range(batches)):
  selected_entities = unique_entities_inductive_testset[i*batch_size: (i+1)*batch_size] 
  selected_entities_string = ' '.join(selected_entities)
  query  =  """
  PREFIX uom: <http://www.opengis.net/def/uom/OGC/1.0/>
  PREFIX wkgs: <http://www.worldkg.org/schema/>
  PREFIX wkg: <http://www.worldkg.org/resource/>
  PREFIX wd: <http://www.wikidata.org/wiki/>
  PREFIX sf: <http://www.opengis.net/ont/sf#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX geo: <http://www.opengis.net/ont/geosparql#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  SELECT DISTINCT ?entity_uri ?type ?label ?nameEn ?coordinates WHERE {
    VALUES ?entity_uri {%s} .
    ?entity_uri wkgs:spatialObject ?geom.
    ?geom geo:asWKT ?coordinates.
    OPTIONAL { ?entity_uri rdf:type ?type .}
    OPTIONAL { ?entity_uri rdfs:label ?label .}
    OPTIONAL { ?entity_uri wkgs:nameEn ?nameEn .}
  }
  """%(selected_entities_string)
  print(query)
  try:
    results = query_sparql(query,sparqlview_wrapper)
  except Exception as e:
    print(sys.exc_info()[0])
    print(e)

  results_df = sparql2dataframe(results)
  print(results_df.shape)
  results_df_list.append(results_df)


merged_results = pd.concat(results_df_list, ignore_index = True)

merged_results

  0%|          | 0/162 [00:00<?, ?it/s]


  PREFIX uom: <http://www.opengis.net/def/uom/OGC/1.0/>
  PREFIX wkgs: <http://www.worldkg.org/schema/>
  PREFIX wkg: <http://www.worldkg.org/resource/>
  PREFIX wd: <http://www.wikidata.org/wiki/>
  PREFIX sf: <http://www.opengis.net/ont/sf#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX geo: <http://www.opengis.net/ont/geosparql#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  SELECT DISTINCT ?entity_uri ?type ?label ?nameEn ?coordinates WHERE {
    VALUES ?entity_uri {wkg:1003705119 wkg:1011817361 wkg:1012761434 wkg:1013550821 wkg:101480194 wkg:1029617830 wkg:1042491447 wkg:1042516435 wkg:1042944919 wkg:1043921912 wkg:1047237485 wkg:1048268052 wkg:1048638884 wkg:1049908636 wkg:1060112701 wkg:1066143903 wkg:1068535143 wkg:1068823637 wkg:1068823679 wkg:1070164380 wkg:1070176513 wkg:1070187263 wkg:1070187603 wkg:1070195044 wkg:1072625851 wkg:1074913897 wkg:1077973943 wkg:1089286303 wkg:1094041525 wkg:10944

  1%|          | 1/162 [00:00<01:38,  1.63it/s]

(100, 5)

  PREFIX uom: <http://www.opengis.net/def/uom/OGC/1.0/>
  PREFIX wkgs: <http://www.worldkg.org/schema/>
  PREFIX wkg: <http://www.worldkg.org/resource/>
  PREFIX wd: <http://www.wikidata.org/wiki/>
  PREFIX sf: <http://www.opengis.net/ont/sf#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX geo: <http://www.opengis.net/ont/geosparql#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  SELECT DISTINCT ?entity_uri ?type ?label ?nameEn ?coordinates WHERE {
    VALUES ?entity_uri {wkg:1150880699 wkg:1150880714 wkg:1150880743 wkg:1150880749 wkg:1150880752 wkg:1150880818 wkg:1150880820 wkg:1150880850 wkg:1150880878 wkg:1150880898 wkg:1150880908 wkg:1150880930 wkg:1150880948 wkg:1150880966 wkg:1150880978 wkg:1150880983 wkg:1150880988 wkg:1150880989 wkg:1150880992 wkg:1150880998 wkg:1150881011 wkg:1150881012 wkg:1150881025 wkg:1150881098 wkg:1150881115 wkg:1150881124 wkg:1150881161 wkg:1150881183 wkg:1150881238

  1%|          | 2/162 [00:01<01:23,  1.91it/s]

...

 99%|█████████▉| 161/162 [01:22<00:00,  1.94it/s]

(100, 5)

  PREFIX uom: <http://www.opengis.net/def/uom/OGC/1.0/>
  PREFIX wkgs: <http://www.worldkg.org/schema/>
  PREFIX wkg: <http://www.worldkg.org/resource/>
  PREFIX wd: <http://www.wikidata.org/wiki/>
  PREFIX sf: <http://www.opengis.net/ont/sf#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX geo: <http://www.opengis.net/ont/geosparql#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  SELECT DISTINCT ?entity_uri ?type ?label ?nameEn ?coordinates WHERE {
    VALUES ?entity_uri {wkg:903016065 wkg:903016390 wkg:903018492 wkg:903018734 wkg:903018868 wkg:903018934 wkg:903019066 wkg:903020520 wkg:903021312 wkg:903021346 wkg:903021625 wkg:903021750 wkg:903021807 wkg:903022517 wkg:903023109 wkg:903023303 wkg:903023539 wkg:903023791 wkg:903023963 wkg:903024075 wkg:904394988 wkg:905640075 wkg:907983288 wkg:912644242 wkg:914675868 wkg:915301419 wkg:919530900 wkg:919531013 wkg:919531167 wkg:919755439 wkg:921136374 

100%|██████████| 162/162 [01:23<00:00,  1.95it/s]

(90, 5)


,entity_uri,type,label,nameEn,coordinates
0,http://www.worldkg.org/resource/1048268052,http://www.worldkg.org/schema/Village,ບ້ານ ສົມສະໄໝ,Ban Somsamay,POINT(102.8220741 18.1366351)
1,http://www.worldkg.org/resource/1048638884,http://www.worldkg.org/schema/Village,Veunthan,Veunthan,POINT(102.8087386 18.1418518)
2,http://www.worldkg.org/resource/1049908636,http://www.worldkg.org/schema/Village,Na Phieng,Na Phieng,POINT(103.5042558 20.0638525)
3,http://www.worldkg.org/resource/1148185540,http://www.worldkg.org/schema/Village,کسرینه,Kasrineh,POINT(58.3997166 35.2567892)
4,http://www.worldkg.org/resource/1013550821,http://www.worldkg.org/schema/Village,سرهنگ,Sarhang,POINT(59.2641824 35.6540583)
...,...,...,...,...,...
16250,http://www.worldkg.org/resource/962290922,http://www.worldkg.org/schema/Museum,Historisches Museum,None,POINT(7.4802326 53.4692332)
16251,http://www.worldkg.org/resource/966583366,http://www.worldkg.org/schema/Town,Kabataş,None,POINT(37.4498616 40.7515493)
16252,http://www.worldkg.org/resource/97160788,http://www.worldkg.org/schema/Town,Lilydale,None,POINT(145.352405 -37.7576639)
16253,http://www.worldkg.org/resource/970564374,http://www.worldkg.org/schema/Village,Gaffarlı,None,POINT(35.9983643 37.5413165)


In [ ]:
# checking and removing duplicates
merged_results = merged_results[~merged_results.duplicated(['entity_uri'], keep = 'first')]
merged_results.shape

(16190, 5)

In [ ]:
# clean uri
pattern = '^http://www.worldkg.org/schema/'
compiled_pattern = re.compile(pattern)

merged_results['type'] = merged_results['type'].astype('str')
merged_results['type'] = merged_results['type'].apply(lambda uri: re.sub(compiled_pattern, 'wkgs:', uri))
merged_results.head()

<ipython-input-14-c492af3911cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_results['type'] = merged_results['type'].astype('str')
<ipython-input-14-c492af3911cb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_results['type'] = merged_results['type'].apply(lambda uri: re.sub(compiled_pattern, 'wkgs:', uri))


,entity_uri,type,label,nameEn,coordinates
0,http://www.worldkg.org/resource/1048268052,wkgs:Village,ບ້ານ ສົມສະໄໝ,Ban Somsamay,POINT(102.8220741 18.1366351)
1,http://www.worldkg.org/resource/1048638884,wkgs:Village,Veunthan,Veunthan,POINT(102.8087386 18.1418518)
2,http://www.worldkg.org/resource/1049908636,wkgs:Village,Na Phieng,Na Phieng,POINT(103.5042558 20.0638525)
3,http://www.worldkg.org/resource/1148185540,wkgs:Village,کسرینه,Kasrineh,POINT(58.3997166 35.2567892)
4,http://www.worldkg.org/resource/1013550821,wkgs:Village,سرهنگ,Sarhang,POINT(59.2641824 35.6540583)


In [ ]:
# replace string 'None'
merged_results.replace(to_replace=['None'], value=np.nan, inplace=True)

<ipython-input-15-a1aa6e7c9182>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_results.replace(to_replace=['None'], value=np.nan, inplace=True)


In [ ]:
# save
## Note: deleted this file, directly saving file with geohashes
merged_results.to_csv('inductive_133testset_entities_data.csv', index = False)

## Add geohash to above csv

In [ ]:
##
# Utilitites code
##
def get_geohash(row):
  '''
  Calculate geohash
  '''
  return gh.encode(row['lat'], row['lon'], precision = 6)

def clean_coordinates(df):
  '''
  :param df - input dataframe
  return clean_df - dataframe with separate columns for 'lat' and 'lon'
  '''
  # applying regex to remove 'Point(' and ')'
  pattern = '(POINT\()|(\))'
  compiled_pattern = re.compile(pattern)
  df['lon_lat'] = df['coordinates'].apply(lambda value: re.sub(compiled_pattern, '', value).split(' '))

  # creating separate columns for lon and lat and convert lat lon columns to float
  df[['lon', 'lat']] = pd.DataFrame(df['lon_lat'].to_list(), index = df.index)
  df['lat'] = df['lat'].astype('float')
  df['lon'] = df['lon'].astype('float')
  
  df.drop(['lon_lat'], axis = 1, inplace = True)

  return df

In [ ]:
# parse coordinates into lat lon column
entities_data_with_geohash = clean_coordinates(merged_results)
entities_data_with_geohash

<ipython-input-16-c643f433bbb7>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lon_lat'] = df['coordinates'].apply(lambda value: re.sub(compiled_pattern, '', value).split(' '))
<ipython-input-16-c643f433bbb7>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lon', 'lat']] = pd.DataFrame(df['lon_lat'].to_list(), index = df.index)
<ipython-input-16-c643f433bbb7>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,entity_uri,type,label,nameEn,coordinates,lon,lat
0,http://www.worldkg.org/resource/1048268052,wkgs:Village,ບ້ານ ສົມສະໄໝ,Ban Somsamay,POINT(102.8220741 18.1366351),102.822074,18.136635
1,http://www.worldkg.org/resource/1048638884,wkgs:Village,Veunthan,Veunthan,POINT(102.8087386 18.1418518),102.808739,18.141852
2,http://www.worldkg.org/resource/1049908636,wkgs:Village,Na Phieng,Na Phieng,POINT(103.5042558 20.0638525),103.504256,20.063852
3,http://www.worldkg.org/resource/1148185540,wkgs:Village,کسرینه,Kasrineh,POINT(58.3997166 35.2567892),58.399717,35.256789
4,http://www.worldkg.org/resource/1013550821,wkgs:Village,سرهنگ,Sarhang,POINT(59.2641824 35.6540583),59.264182,35.654058
...,...,...,...,...,...,...,...
16250,http://www.worldkg.org/resource/962290922,wkgs:Museum,Historisches Museum,None,POINT(7.4802326 53.4692332),7.480233,53.469233
16251,http://www.worldkg.org/resource/966583366,wkgs:Town,Kabataş,None,POINT(37.4498616 40.7515493),37.449862,40.751549
16252,http://www.worldkg.org/resource/97160788,wkgs:Town,Lilydale,None,POINT(145.352405 -37.7576639),145.352405,-37.757664
16253,http://www.worldkg.org/resource/970564374,wkgs:Village,Gaffarlı,None,POINT(35.9983643 37.5413165),35.998364,37.541317


In [ ]:
tqdm.pandas()
entities_data_with_geohash['geohash'] = entities_data_with_geohash.progress_apply(get_geohash, axis = 1)
entities_data_with_geohash

100%|██████████| 16190/16190 [00:00<00:00, 39117.14it/s]
<ipython-input-18-fc21bd0199d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entities_data_with_geohash['geohash'] = entities_data_with_geohash.progress_apply(get_geohash, axis = 1)


,entity_uri,type,label,nameEn,coordinates,lon,lat,geohash
0,http://www.worldkg.org/resource/1048268052,wkgs:Village,ບ້ານ ສົມສະໄໝ,Ban Somsamay,POINT(102.8220741 18.1366351),102.822074,18.136635,w71p5v
1,http://www.worldkg.org/resource/1048638884,wkgs:Village,Veunthan,Veunthan,POINT(102.8087386 18.1418518),102.808739,18.141852,w71p5q
2,http://www.worldkg.org/resource/1049908636,wkgs:Village,Na Phieng,Na Phieng,POINT(103.5042558 20.0638525),103.504256,20.063852,w79d5k
3,http://www.worldkg.org/resource/1148185540,wkgs:Village,کسرینه,Kasrineh,POINT(58.3997166 35.2567892),58.399717,35.256789,tq388f
4,http://www.worldkg.org/resource/1013550821,wkgs:Village,سرهنگ,Sarhang,POINT(59.2641824 35.6540583),59.264182,35.654058,tq64ud
...,...,...,...,...,...,...,...,...
16250,http://www.worldkg.org/resource/962290922,wkgs:Museum,Historisches Museum,None,POINT(7.4802326 53.4692332),7.480233,53.469233,u1t24j
16251,http://www.worldkg.org/resource/966583366,wkgs:Town,Kabataş,None,POINT(37.4498616 40.7515493),37.449862,40.751549,sz4xu4
16252,http://www.worldkg.org/resource/97160788,wkgs:Town,Lilydale,None,POINT(145.352405 -37.7576639),145.352405,-37.757664,r1r35w
16253,http://www.worldkg.org/resource/970564374,wkgs:Village,Gaffarlı,None,POINT(35.9983643 37.5413165),35.998364,37.541317,sy9te4


In [ ]:
# create a column to indicate cluster assignment for each precision
entities_data_with_geohash['1_precision_cluster_center'] = entities_data_with_geohash['geohash'].apply(lambda x: x[0])
entities_data_with_geohash['3_precision_cluster_center'] = entities_data_with_geohash['geohash'].apply(lambda x: x[:3])
entities_data_with_geohash['4_precision_cluster_center'] = entities_data_with_geohash['geohash'].apply(lambda x: x[:4])
entities_data_with_geohash

<ipython-input-19-369369fad540>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entities_data_with_geohash['1_precision_cluster_center'] = entities_data_with_geohash['geohash'].apply(lambda x: x[0])
<ipython-input-19-369369fad540>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entities_data_with_geohash['3_precision_cluster_center'] = entities_data_with_geohash['geohash'].apply(lambda x: x[:3])
<ipython-input-19-369369fad540>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,entity_uri,type,label,nameEn,coordinates,lon,lat,geohash,1_precision_cluster_center,3_precision_cluster_center,4_precision_cluster_center
0,http://www.worldkg.org/resource/1048268052,wkgs:Village,ບ້ານ ສົມສະໄໝ,Ban Somsamay,POINT(102.8220741 18.1366351),102.822074,18.136635,w71p5v,w,w71,w71p
1,http://www.worldkg.org/resource/1048638884,wkgs:Village,Veunthan,Veunthan,POINT(102.8087386 18.1418518),102.808739,18.141852,w71p5q,w,w71,w71p
2,http://www.worldkg.org/resource/1049908636,wkgs:Village,Na Phieng,Na Phieng,POINT(103.5042558 20.0638525),103.504256,20.063852,w79d5k,w,w79,w79d
3,http://www.worldkg.org/resource/1148185540,wkgs:Village,کسرینه,Kasrineh,POINT(58.3997166 35.2567892),58.399717,35.256789,tq388f,t,tq3,tq38
4,http://www.worldkg.org/resource/1013550821,wkgs:Village,سرهنگ,Sarhang,POINT(59.2641824 35.6540583),59.264182,35.654058,tq64ud,t,tq6,tq64
...,...,...,...,...,...,...,...,...,...,...,...
16250,http://www.worldkg.org/resource/962290922,wkgs:Museum,Historisches Museum,None,POINT(7.4802326 53.4692332),7.480233,53.469233,u1t24j,u,u1t,u1t2
16251,http://www.worldkg.org/resource/966583366,wkgs:Town,Kabataş,None,POINT(37.4498616 40.7515493),37.449862,40.751549,sz4xu4,s,sz4,sz4x
16252,http://www.worldkg.org/resource/97160788,wkgs:Town,Lilydale,None,POINT(145.352405 -37.7576639),145.352405,-37.757664,r1r35w,r,r1r,r1r3
16253,http://www.worldkg.org/resource/970564374,wkgs:Village,Gaffarlı,None,POINT(35.9983643 37.5413165),35.998364,37.541317,sy9te4,s,sy9,sy9t


In [ ]:
# entities_data_with_geohash.to_csv('inductive_133testset_entities_data_with_geohash_cluster_assignment.csv', index = False)
entities_data_with_geohash.to_csv('split2testset_entities_data_with_geohash_cluster_assignment.csv', index = False)

In [ ]:
def pairwise_similarity_from_distance_matrix(distance_matrix):
  '''
  :param distance_matrix: distance_matrix of cluster centers
  :return pairwise similarity matrix based on distance matrix with values in [0, 1]
  '''
  
  # convert matrix of distances to values between 0-1 and change distance to similarity
  # i.r d/d_max and the 1 - d/d_max
  # pairwise_similarity_matrix = distance_matrix / np.broadcast_to(distance_matrix.max(axis = 0), distance_matrix.shape)
  # pairwise_similarity_matrix = 1 - pairwise_similarity_matrix
  
  # return pairwise_similarity_matrix
  return 1 - (distance_matrix / np.broadcast_to(distance_matrix.max(axis = 0), distance_matrix.shape))

In [ ]:
precision1_cluster_center_geohashes = np.unique(entities_data_with_geohash['1_precision_cluster_center'])
precision1_cluster_lat_lon_list = [(gh.decode(hash)[0],  gh.decode(hash)[1]) for hash in precision1_cluster_center_geohashes]
precision1_cluster_distance_matrix = haversine_vector(
    precision1_cluster_lat_lon_list,
    precision1_cluster_lat_lon_list,
    Unit.KILOMETERS,
    comb=True
)


precision3_cluster_center_geohashes = np.unique(entities_data_with_geohash['3_precision_cluster_center'])
precision3_cluster_lat_lon_list = [(gh.decode(hash)[0],  gh.decode(hash)[1]) for hash in precision3_cluster_center_geohashes]
precision3_cluster_distance_matrix = haversine_vector(
    precision3_cluster_lat_lon_list,
    precision3_cluster_lat_lon_list,
    Unit.KILOMETERS,
    comb=True
)


In [ ]:
precision1_similarity_matrix = pairwise_similarity_from_distance_matrix(precision1_cluster_distance_matrix)
precision3_similarity_matrix = pairwise_similarity_from_distance_matrix(precision3_cluster_distance_matrix)
# save embedding array
with h5py.File('precision1_similarity_matrix_file.h5', 'w') as hf:
  hf.create_dataset('precision1_similarity_matrix',  data = precision1_similarity_matrix, compression = 'gzip')

with h5py.File('precision3_similarity_matrix_file.h5', 'w') as hf:
  hf.create_dataset('precision3_similarity_matrix',  data = precision3_similarity_matrix, compression = 'gzip')

In [ ]:
precision4_cluster_center_geohashes = np.unique(entities_data_with_geohash['4_precision_cluster_center'])
precision4_cluster_lat_lon_list = [(gh.decode(hash)[0],  gh.decode(hash)[1]) for hash in precision4_cluster_center_geohashes]
precision4_cluster_distance_matrix = haversine_vector(
    precision4_cluster_lat_lon_list,
    precision4_cluster_lat_lon_list,
    Unit.KILOMETERS,
    comb=True
)
precision4_cluster_distance_matrix = precision4_cluster_distance_matrix.astype(
    dtype = np.float16,
    copy = False
    )
precision4_similarity_matrix = pairwise_similarity_from_distance_matrix(precision4_cluster_distance_matrix)
with h5py.File('precision4_similarity_matrix_file.h5', 'w') as hf:
  hf.create_dataset('precision4_similarity_matrix',  data = precision4_similarity_matrix, compression = 'gzip')

## FT mean label_nameEn embeddings

In [ ]:
# load fastText model
fasttext.util.download_model('en', if_exists = 'ignore')  # English
model = fasttext.load_model('cc.en.300.bin')

In [ ]:
'''
Utility to return fastText embedding of word x
using fastText and for <UNK> token return zero vector
'''
def get_ft_embedding(x, model):
  if x == '<UNK>':
    return np.zeros(300)
  else:
    return model.get_word_vector(str(x).lower())

In [ ]:
mean_ft_embedding = np.zeros((entities_data_with_geohash.shape[0], 300), dtype = np.float32)

for i, label, nameEn in tqdm(zip(range(entities_data_with_geohash.shape[0]),entities_data_with_geohash['label'], entities_data_with_geohash['nameEn'])):

  mean_ft_embedding[i] = np.mean(np.concatenate((
      get_ft_embedding(label, model).reshape(1, -1),
      get_ft_embedding(nameEn, model).reshape(1, -1)
  )), axis = 0)

mean_ft_embedding.shape

16190it [00:00, 16493.73it/s]


(16190, 300)

In [ ]:
# save embedding
with h5py.File('split2_mean_label_nameEn_embeddings_file.h5', 'w') as hf:
  hf.create_dataset('mean_label_nameEn_embeddings',  data = mean_ft_embedding, compression = 'gzip')

In [ ]:
# label entity_idx dict
entity_uris = entities_data_with_geohash['entity_uri'].values
entity_idx = dict()
for i, uri in zip(range(len(entity_uris)), entity_uris):
  entity_idx[uri] = i
  
with open('label_entity_idx.pickle', 'wb') as handle:
  pickle.dump(entity_idx, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Type and relation embeddings

In [ ]:
# entities_data_with_geohash = pd.read_csv(
#     'inductive_133testset_entities_data_with_geohash_cluster_assignment.csv'
#     )
entities_data_with_geohash = pd.read_csv(
    'split2testset_entities_data_with_geohash_cluster_assignment.csv'
)
entities_data_with_geohash

,entity_uri,type,label,nameEn,coordinates,lon,lat,geohash,1_precision_cluster_center,3_precision_cluster_center,4_precision_cluster_center
0,http://www.worldkg.org/resource/1048268052,wkgs:Village,ບ້ານ ສົມສະໄໝ,Ban Somsamay,POINT(102.8220741 18.1366351),102.822074,18.136635,w71p5v,w,w71,w71p
1,http://www.worldkg.org/resource/1048638884,wkgs:Village,Veunthan,Veunthan,POINT(102.8087386 18.1418518),102.808739,18.141852,w71p5q,w,w71,w71p
2,http://www.worldkg.org/resource/1049908636,wkgs:Village,Na Phieng,Na Phieng,POINT(103.5042558 20.0638525),103.504256,20.063852,w79d5k,w,w79,w79d
3,http://www.worldkg.org/resource/1148185540,wkgs:Village,کسرینه,Kasrineh,POINT(58.3997166 35.2567892),58.399717,35.256789,tq388f,t,tq3,tq38
4,http://www.worldkg.org/resource/1013550821,wkgs:Village,سرهنگ,Sarhang,POINT(59.2641824 35.6540583),59.264182,35.654058,tq64ud,t,tq6,tq64
...,...,...,...,...,...,...,...,...,...,...,...
16185,http://www.worldkg.org/resource/962290922,wkgs:Museum,Historisches Museum,NaN,POINT(7.4802326 53.4692332),7.480233,53.469233,u1t24j,u,u1t,u1t2
16186,http://www.worldkg.org/resource/966583366,wkgs:Town,Kabataş,NaN,POINT(37.4498616 40.7515493),37.449862,40.751549,sz4xu4,s,sz4,sz4x
16187,http://www.worldkg.org/resource/97160788,wkgs:Town,Lilydale,NaN,POINT(145.352405 -37.7576639),145.352405,-37.757664,r1r35w,r,r1r,r1r3
16188,http://www.worldkg.org/resource/970564374,wkgs:Village,Gaffarlı,NaN,POINT(35.9983643 37.5413165),35.998364,37.541317,sy9te4,s,sy9,sy9t


In [ ]:
# fill unknown type with <UNK>
entities_data_with_geohash['type'].fillna('<UNK>', inplace = True)

In [ ]:
##
# !NOTE create dictionary with prefixes wkgs:
##
# list of unique types
unique_types, type_counts = np.unique(entities_data_with_geohash[['type']].values.flatten(), return_counts = True)


# create unique_types to idx dictionary
type_to_idx_dict = dict()
for i in range(len(unique_types)):
  type_to_idx_dict[unique_types[i]] = i
type_to_idx_dict

{'<UNK>': 0,
 'wkgs:Administrative': 1,
 'wkgs:AerialwayStation': 2,
 'wkgs:Aerodrome': 3,
 'wkgs:Alcohol': 4,
 'wkgs:AlpineHut': 5,
 'wkgs:AmenityCollege': 6,
 'wkgs:AmenityFuel': 7,
 'wkgs:AmenityHospital': 8,
 'wkgs:AmenityIceCream': 9,
 'wkgs:AmenitySchool': 10,
 'wkgs:AmenityUniversity': 11,
 'wkgs:Aquarium': 12,
 'wkgs:ArchaeologicalSite': 13,
 'wkgs:Archipelago': 14,
 'wkgs:Art': 15,
 'wkgs:ArtsCentre': 16,
 'wkgs:Association': 17,
 'wkgs:Atm': 18,
 'wkgs:Attraction': 19,
 'wkgs:Bank': 20,
 'wkgs:Bar': 21,
 'wkgs:BloodDonation': 22,
 'wkgs:Books': 23,
 'wkgs:Brewery': 24,
 'wkgs:Building': 25,
 'wkgs:BuildingBunker': 26,
 'wkgs:BuildingChurch': 27,
 'wkgs:BuildingCommercial': 28,
 'wkgs:BuildingResidential': 29,
 'wkgs:BuildingSchool': 30,
 'wkgs:Cafe': 31,
 'wkgs:Car': 32,
 'wkgs:Carpenter': 33,
 'wkgs:Casino': 34,
 'wkgs:Castle': 35,
 'wkgs:Chapel': 36,
 'wkgs:ChargingStation': 37,
 'wkgs:Cinema': 38,
 'wkgs:City': 39,
 'wkgs:CityGate': 40,
 'wkgs:Cliff': 41,
 'wkgs:Climbing':

In [ ]:
entities_data_with_geohash[entities_data_with_geohash['type'] == 'http://www.worldkg.org/schema/Butcher']

,entity_uri,type,label,nameEn,coordinates,lon,lat,geohash,1_precision_cluster_center,3_precision_cluster_center,4_precision_cluster_center


In [ ]:
# remove 'wkgs:' prefix from type
pattern = '^wkgs:'
compiled_pattern = re.compile(pattern)

# merged_results['type'] = merged_results['type'].astype('str')
entities_data_with_geohash['type'] = entities_data_with_geohash['type'].apply(lambda uri: re.sub(compiled_pattern, '', uri))
entities_data_with_geohash.head()

,entity_uri,type,label,nameEn,coordinates,lon,lat,geohash,1_precision_cluster_center,3_precision_cluster_center,4_precision_cluster_center
0,http://www.worldkg.org/resource/1048268052,Village,ບ້ານ ສົມສະໄໝ,Ban Somsamay,POINT(102.8220741 18.1366351),102.822074,18.136635,w71p5v,w,w71,w71p
1,http://www.worldkg.org/resource/1048638884,Village,Veunthan,Veunthan,POINT(102.8087386 18.1418518),102.808739,18.141852,w71p5q,w,w71,w71p
2,http://www.worldkg.org/resource/1049908636,Village,Na Phieng,Na Phieng,POINT(103.5042558 20.0638525),103.504256,20.063852,w79d5k,w,w79,w79d
3,http://www.worldkg.org/resource/1148185540,Village,کسرینه,Kasrineh,POINT(58.3997166 35.2567892),58.399717,35.256789,tq388f,t,tq3,tq38
4,http://www.worldkg.org/resource/1013550821,Village,سرهنگ,Sarhang,POINT(59.2641824 35.6540583),59.264182,35.654058,tq64ud,t,tq6,tq64


In [ ]:
unique_types, type_counts = np.unique(entities_data_with_geohash[['type']].values.flatten(), return_counts = True)

In [ ]:
# merging relation names with types to embed both in same space,
# but we are maintaining a separate dictionary for relation indices
relation_name_idx_dict = {
    'wkgs:isIn': 0,
    'wkgs:addrPlace': 1,
    'wkgs:isInContinent': 2,
    'wkgs:country': 3,
    'wkgs:isInCountry': 4,
    'wkgs:addrCountry': 5,
    'wkgs:capitalCity': 6,
    'wkgs:addrState': 7,
    'wkgs:addrDistrict': 8,
    'wkgs:addrProvince': 9,
    'wkgs:isInCounty': 10,
    'wkgs:addrSubdistrict': 11,
    'wkgs:addrSuburb': 12,
    'wkgs:addrHamlet': 13
}

# update indices as continuation from rdf:type indices
index_start = len(unique_types)
relation_names = []
for key, val in relation_name_idx_dict.items():
  relation_name_idx_dict[key] = val + index_start
  relation_names.append(key)

In [ ]:
relation_name_idx_dict

{'wkgs:isIn': 130,
 'wkgs:addrPlace': 131,
 'wkgs:isInContinent': 132,
 'wkgs:country': 133,
 'wkgs:isInCountry': 134,
 'wkgs:addrCountry': 135,
 'wkgs:capitalCity': 136,
 'wkgs:addrState': 137,
 'wkgs:addrDistrict': 138,
 'wkgs:addrProvince': 139,
 'wkgs:isInCounty': 140,
 'wkgs:addrSubdistrict': 141,
 'wkgs:addrSuburb': 142,
 'wkgs:addrHamlet': 143}

In [ ]:
# remove 'wkgs:' prefix from relation name
pattern = '^wkgs:'
compiled_pattern = re.compile(pattern)

# merged_results['type'] = merged_results['type'].astype('str')
relation_names = [re.sub(compiled_pattern, '', uri) for uri in relation_names]

In [ ]:
unique_types

array(['<UNK>', 'Administrative', 'AerialwayStation', 'Aerodrome',
       'Alcohol', 'AlpineHut', 'AmenityCollege', 'AmenityFuel',
       'AmenityHospital', 'AmenityIceCream', 'AmenitySchool',
       'AmenityUniversity', 'Aquarium', 'ArchaeologicalSite',
       'Archipelago', 'Art', 'ArtsCentre', 'Association', 'Atm',
       'Attraction', 'Bank', 'Bar', 'BloodDonation', 'Books', 'Brewery',
       'Building', 'BuildingBunker', 'BuildingChurch',
       'BuildingCommercial', 'BuildingResidential', 'BuildingSchool',
       'Cafe', 'Car', 'Carpenter', 'Casino', 'Castle', 'Chapel',
       'ChargingStation', 'Cinema', 'City', 'CityGate', 'Cliff',
       'Climbing', 'Clinic', 'Clothes', 'Company', 'Continent', 'Country',
       'County', 'Courthouse', 'Deli', 'Diplomatic', 'District',
       'EducationalInstitution', 'Electronics', 'FastFood', 'FieldHockey',
       'Flagpole', 'Foundation', 'Fountain', 'Gallery', 'GuestHouse',
       'Hackerspace', 'Halt', 'Hamlet', 'HistoricMilestone',
      

In [ ]:
# merge relation names with type
types_and_relation_names = np.concatenate((unique_types, np.array(relation_names)))
types_and_relation_names.shape

# generate embeddings stored at corresponding idx
embeddings = np.array([get_ft_embedding(x, model) for x in types_and_relation_names], dtype = np.float64)

In [ ]:
# save as h5py
with h5py.File('type_and_relation_embeddings_file.h5', 'w') as hf:
  hf.create_dataset('type_embeddings',  data = embeddings, compression = 'gzip')

In [ ]:
# save type index as .dict file
with open('type_to_idx_dict.dict', 'wb') as handle:
    pickle.dump(type_to_idx_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# save relation_name index as .dict file
with open('relation_name_idx_dict.dict', 'wb') as handle:
    pickle.dump(relation_name_idx_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## FT literal value embeddings

In [ ]:
# test_inductive = pd.read_csv('test_inductive_133triples.txt', sep = '\t', names = ['subject', 'predicate', 'object'])
# test_inductive
test_inductive = pd.read_csv('test_filtered_triples_deduplicated.txt', sep = '\t', names = ['subject', 'predicate', 'object'])
test_inductive

,subject,predicate,object
0,wkg:4426850370,wkgs:isInCountry,wkg:424298311
1,wkg:3068656670,wkgs:isInCountry,wkg:424313730
2,wkg:630639045,wkgs:isInContinent,wkg:36966065
3,wkg:4426852196,wkgs:isInContinent,wkg:36966065
4,wkg:2000003612,wkgs:isInCountry,wkg:424298311
...,...,...,...
15926,wkg:274372610,wkgs:addrCountry,wkg:1683325355
15927,wkg:1254881907,wkgs:addrCountry,wkg:424314830
15928,wkg:1261054393,wkgs:addrCountry,wkg:424314830
15929,wkg:279000168,wkgs:addrCountry,wkg:6907263340


In [ ]:
spo_literal_value_df = pd.read_csv(
    'inductive_spo_plus_literal.csv',
    names = ['subject', 'predicate', 'object', 'literal_object']
    )
spo_literal_value_df

,subject,predicate,object,literal_object
0,wkg:4426850370,wkgs:isInCountry,wkg:424298311,Iran
1,wkg:1923575883,wkgs:isInContinent,wkg:36966065,Asia
2,wkg:1308683858,wkgs:isInCountry,wkg:6677037562,Indonesia
3,wkg:1308609726,wkgs:isInCountry,wkg:6677037562,Indonesia
4,wkg:3068656670,wkgs:isInCountry,wkg:424313730,Bolivia
...,...,...,...,...
159991,wkg:1135414237,wkgs:addrCountry,wkg:424314830,RU
159992,wkg:1197106724,wkgs:addrCountry,wkg:424314830,RU
159993,wkg:945163900,wkgs:addrCountry,wkg:424314830,RU
159994,wkg:961947169,wkgs:addrCountry,wkg:424314830,RU


In [ ]:
# merge test_inductive with spo_literal_value_df
test_inductive_with_literals = test_inductive.merge(
    spo_literal_value_df,
    on = ['subject', 'predicate', 'object'],
    how = 'left'
).drop_duplicates().reset_index(drop = True)

test_inductive_with_literals

,subject,predicate,object,literal_object
0,wkg:4437682914,wkgs:addrDistrict,wkg:4310758164,城关区
1,wkg:7800531449,wkgs:addrDistrict,wkg:316417378,Prakasam
2,wkg:4165125720,wkgs:addrDistrict,wkg:4310758164,城关区
3,wkg:5683455875,wkgs:addrHamlet,wkg:338530778,Moeraki
4,wkg:1600794872,wkgs:addrSubdistrict,wkg:634183692,เขาสวนกวาง
...,...,...,...,...
128,wkg:8505872425,wkgs:isInCounty,wkg:6715794617,平陆县
129,wkg:26984576,wkgs:isInCounty,wkg:302323945,Derbyshire
130,wkg:871196365,wkgs:isInCounty,wkg:316944811,Los Angeles
131,wkg:7831296688,wkgs:isInCounty,wkg:6618706932,雄县


In [ ]:
test_inductive_with_literals[pd.isna(test_inductive_with_literals['literal_object'])]

,subject,predicate,object,literal_object
26,wkg:862872811,wkgs:addrCountry,wkg:424314830,NaN


In [ ]:
list(test_inductive_with_literals['subject'].values) == list(test_inductive['subject'].values)

True

In [ ]:
##
# Change for split 2 - performed literal value merge in another csv
##
test_inductive_with_literals = pd.read_csv('test_filtered_triples_deduplicated_with_literal_object.csv')
test_inductive_with_literals

,subject,predicate,object,literal_object
0,wkg:4426850370,wkgs:isInCountry,wkg:424298311,Iran
1,wkg:3068656670,wkgs:isInCountry,wkg:424313730,Bolivia
2,wkg:630639045,wkgs:isInContinent,wkg:36966065,Asia
3,wkg:4426852196,wkgs:isInContinent,wkg:36966065,Asia
4,wkg:2000003612,wkgs:isInCountry,wkg:424298311,Iran
...,...,...,...,...
15926,wkg:274372610,wkgs:addrCountry,wkg:1683325355,DE
15927,wkg:1254881907,wkgs:addrCountry,wkg:424314830,RU
15928,wkg:1261054393,wkgs:addrCountry,wkg:424314830,RU
15929,wkg:279000168,wkgs:addrCountry,wkg:6907263340,DE


In [ ]:
###
# EMBEDDING STEP
###
# triple idx corresponds to literal embedding idx
# FT embedding of literal values
_property_name = 'literal_object'
property_numpy = test_inductive_with_literals[_property_name].to_numpy(copy = True)
literal_value_embeddings = np.array([get_ft_embedding(x, model) for x in property_numpy], dtype = np.float64)
literal_value_embeddings.shape

(15931, 300)

In [ ]:
# save as h5py
with h5py.File('split2test_set_literal_value_embeddings_file.h5', 'w') as hf:
    hf.create_dataset('test_set_literal_value_embeddings',  data = literal_value_embeddings, compression = 'gzip')

## Mean label, nameEn embedding

In [ ]:
_property_name = 'label' #nameEn
entities_data_with_geohash[_property_name].fillna('<UNK>', inplace = True)
entities_data_with_geohash[pd.isna(entities_data_with_geohash[_property_name])]

,entity_uri,type,label,nameEn,coordinates,lon,lat,geohash,1_precision_cluster_center,3_precision_cluster_center,4_precision_cluster_center


In [ ]:
property_numpy = entities_data_with_geohash[_property_name].to_numpy(copy = True)

nameEn_embeddings = np.array([get_ft_embedding(x, model) for x in property_numpy], dtype = np.float64)

In [ ]:
property_numpy = entities_data_with_geohash[_property_name].to_numpy(copy = True)

label_embeddings = np.array([get_ft_embedding(x, model) for x in property_numpy], dtype = np.float64)

In [ ]:
nameEn_embeddings.shape

(231, 300)

In [ ]:
label_embeddings.shape

(231, 300)

In [ ]:
(np.add(nameEn_embeddings, label_embeddings) / 2.0).shape

(231, 300)

In [ ]:
mean_label_nameEn_embeddings.shape

(231, 300)

In [ ]:
# save as h5py
with h5py.File('mean_label_nameEn_embeddings_file.h5', 'w') as hf:
    hf.create_dataset('mean_label_nameEn_embeddings',  data = mean_label_nameEn_embeddings, compression = 'gzip')